In [6]:
import os
from tqdm import tqdm_notebook as tqdm
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.neighbors import KDTree
from mpl_toolkits.mplot3d import Axes3D, art3d
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import torch
import torch.nn as nn

import numpy as np

from plyfile import PlyData, PlyElement

In [7]:
def read_mesh(filename):
    
    plydata = PlyData.read(filename)
    vertices = np.vstack((plydata['vertex'].data['x'], plydata['vertex'].data['y'], plydata['vertex'].data['z'])).T
    vert_colors = np.vstack((plydata["vertex"]['red'], 
                             plydata["vertex"]['green'], 
                             plydata["vertex"]['blue'], 
                             plydata["vertex"]['alpha'])).T
    tri_data = plydata['face'].data['vertex_indices']
    faces = np.vstack(tri_data)
    
    face_colors = vert_colors[faces].mean(axis=1) / 255
#     fld_tree = KDTree(vertices)
#     distances, faces = fld_tree.query(vertices, k=3)
    
    return vertices, faces, face_colors


def plot_model_3d(vertices, faces, face_colors, loss, elev, azim, filename, points=None, title=True):
    """
    Visualize mesh object. 
    """

    fig = plt.figure(figsize=(15, 15))
    ax = fig.add_subplot(111, projection='3d')
    
    pc = art3d.Poly3DCollection(vertices[faces],linewidths=0.1, facecolors = face_colors)
    ax.add_collection(pc)
    
    if points is not None:
        ax.scatter(points[:,0], points[:,1], points[:,2], c='red', alpha = 0.2)
    
    ax.set_xlim(-1.0, 1.0)
    ax.set_ylim(-1.0, 1.0)
    ax.set_zlim(-1.0, 1.0)
    
    if title:
        ax.set_title('Objective: '+str(loss), fontdict={'fontsize': 20, 'fontweight': 'medium'})
    ax.view_init(elev=elev, azim = azim)
    #print("storing ", filename)
    plt.savefig(filename, bbox_inches='tight')
    plt.close()
    
def plot_interp(vertices, faces, elev,azim, filename):
    """
    Visualize mesh object. 
    """

    fig = plt.figure(figsize=(15, 15))
    ax = fig.add_subplot(111, projection='3d')
    
    pc = art3d.Poly3DCollection(vertices[faces],linewidths=0.1,edgecolor = 'k')
    ax.add_collection(pc)
    
    
    ax.set_xlim(-1.0, 1.0)
    ax.set_ylim(-1.0, 1.0)
    ax.set_zlim(-1.0, 1.0)
    
    ax.view_init(elev=elev, azim = azim)
    #print("storing ", filename)
    plt.savefig(filename, bbox_inches='tight')
    plt.close()

In [8]:
def process_folder(folder_name, need_continue=False, max_iters=400, env=30):

    if  not os.path.exists(folder_name):
        raise
    else:
        if not os.path.exists(folder_name + '/frames'):
            os.makedirs(folder_name + '/frames')
        if not os.path.exists(folder_name + '/report'):
            os.makedirs(folder_name + '/report')
            
#     losses = np.load(folder_name +"/loss_plot.npy")
#     plt.figure()
#     #plt.yscale("log")
#     plt.xlabel('Iteration')
#     plt.ylabel('Loss')
#     plt.plot(losses)
#     plt.savefig(folder_name + '/report/loss_plot.png')
#     plt.show()

#     latent = np.load(folder_name +"/latent_dist.npy")
#     plt.figure()
#     plt.xlabel('Iteration')
#     plt.ylabel('Distance in latent space')
#     plt.plot(latent)
#     plt.savefig(folder_name + '/report/latent_dist.png')
#     plt.show()

#     lr = np.load(folder_name +"/lr_plot.npy")
#     plt.figure()
#     plt.xlabel('Iteration')
#     plt.ylabel('Learning Rate')
#     plt.plot(lr)
#     plt.savefig(folder_name + '/report/lr_plot.png')
#     plt.show()
    
#     pts = np.load(folder_name +"/target_verts.npy")
#     pts = pts[torch.randperm(pts.shape[0])]
#     pts = pts[0:5000, :]
#     pts = pts[:,[0,2,1]]

    losses = np.load(folder_name +"/loss_plot.npy", allow_pickle=True)

    number_of_frames = len([name for name in os.listdir(folder_name + "/meshes") if name[-4:]=='.ply'])
    number_of_frames = min(number_of_frames, max_iters)
    
    iter_from = 0
    if need_continue:
        iter_from = len([name for name in os.listdir(folder_name + "/frames") if name[-4:]=='.png'])
    
    for i in tqdm(range(iter_from, number_of_frames)):
        vertices, faces, face_colors = read_mesh(folder_name +'/meshes/'+ str(i).zfill(5) +'.ply')
        plot_model_3d(vertices[:,[0,2,1]], faces, face_colors, losses[i], env, 20, folder_name + \
                                                      '/frames/' + str(i).zfill(5)+'.png')
        plot_model_3d(vertices[:,[0,2,1]], faces, face_colors, losses[i], 0, 90, folder_name + \
                                                      '/frames/' + str(i).zfill(5)+'z.png', title=False)
        plot_model_3d(vertices[:,[0,2,1]], faces, face_colors, losses[i], 90, 0, folder_name + \
                                                      '/frames/' + str(i).zfill(5)+'y.png', title=False)

In [9]:
process_folder("../Expirements/SavedTransforms/DeepSDF-CFD8-m3-holes/", need_continue=False)

In [ ]:
process_folder("Expirements/Optimization/VertexDragNew/", need_continue=False)

In [12]:
process_folder("Expirements/Optimization/BboxNew/", need_continue=False)

In [ ]:
process_folder("../Expirements/SavedTransforms/DeepSDF-CFD8-m3-hard-active", need_continue=False)

In [ ]:
process_folder("../Expirements/SavedTransforms/0-18_usual", need_continue=True)

In [ ]:
process_folder("../Expirements/SavedTransforms/0-13_usual", need_continue=False)

In [ ]:
process_folder("../Expirements/SavedTransforms/0-99_usual", need_continue=False)

In [ ]:
process_folder("../Expirements/SavedTransforms/0-18_reindex_weak", need_continue=False)

In [ ]:
process_folder("../Expirements/SavedTransforms/0-13_reindex_weak", need_continue=False)

In [ ]:
process_folder("../Expirements/SavedTransforms/0-18_reindex_strong", need_continue=False)

In [ ]:
process_folder("../Expirements/SavedTransforms/0-18_super_weak", need_continue=False)

In [ ]:
process_folder("../Expirements/SavedTransforms/0-13_reindex_weak_hp", need_continue=False)

In [ ]:
process_folder("../Expirements/SavedTransforms/0-18_reindex_weak_hp", need_continue=False)